In [1]:
### UTILS
def get_input(dayNum):
    import os
    data_file = f"./data/day-{dayNum}.input"
    if not os.path.exists(data_file):
        download_input(dayNum) 
    return [line.strip() for line in open(data_file).readlines()]

def download_input(dayNum):
    import subprocess
    SESSION_COOKIE = open('./session_cookie.secret').readlines()[0].strip()
    url = f"https://adventofcode.com/2020/day/{dayNum}/input"
    cmd = f"curl -H 'Cookie: session={SESSION_COOKIE}' {url} > ./data/day-{dayNum}.input"
    subprocess.run(cmd, capture_output=True, shell=True)

def manhattan_distance(pointA,pointB):
    return abs(pointA[0] - pointB[0]) + abs(pointA[1] - pointB[1])
    
# input is e.g. [0,1,0,0]
# output is e.g 0b0100 -> 4
def bits_to_int(bits):
    return int("0b" + "".join([str(b) for b in bits]), base=2)

assert(bits_to_int([0,1,0,0]) == 4)

# https://en.wikipedia.org/wiki/Extended_Euclidean_algorithm#Example
def extended_gcd(a, b):
    (old_r, r) = (a,b)
    (old_s, s) = (1,0)
    (old_t,t)  = (0,1)

    i = -1
    while r != 0:
        i += 1
        quotient = old_r // r
        (old_r, r) = (r, old_r - quotient * r)
        (old_s, s) = (s, old_s - quotient * s)
        (old_t, t) = (t, old_t - quotient * t)
    
    return {'bezout': (old_s, old_t), 'gcd': old_r, 'quotients': (t, s)}

def bezout_coeff(a,b):
    return extended_gcd(a,b)['bezout']

def split_list(seq, sep=''):
    chunk = []
    for v in seq:
        if v == sep:
            if len(chunk) > 0: yield chunk
            chunk = []
        else:
            chunk.append(v)
    if len(chunk) > 0: yield chunk
    
assert list(split_list(['a','b','','d','f'])) == [ ['a','b'], ['d','f']]
assert list(split_list(['a','b','d','f'])) == [ ['a','b','d','f']]
assert list(split_list(['a','b','d','f',''])) == [ ['a','b','d','f'] ]
assert list(split_list(['','a','b','d','f',''])) == [ ['a','b','d','f'] ]

def findindex(haystack, needle_fn):
    for idx,v in enumerate(haystack):
        if needle_fn(v):
            return idx
        
assert findindex([1,2,3,4], lambda x: 8 -x == 4) == [1,2,3,4].index(4)

In [2]:
# https://adventofcode.com/2020/day/4
def Day4():
    def get_passports():
        passports = []
        lines = get_input(4)
        cur_passport = {}
        for line in lines:
            if line == "":
                if cur_passport:
                    passports.append(cur_passport)
                cur_passport = {}
            field_pairs = line.split()
            for pair in field_pairs:
                [key,val] = pair.split(':')
                cur_passport[key] = val
        passports.append(cur_passport)
        return passports
        
    passports = get_passports()
    
    def valid_1(passport):
        required_fields = ['byr', 'iyr', 'eyr', 'hgt', 'hcl', 'ecl', 'pid']
        if not all([key in passport for key in required_fields]):
            return False
        return True

    def valid_2(passport):
        import re
        if not valid_1(passport):
            return False

        def validate_height(x):
            match = re.match("(\d+)(cm|in)", x)
            if match is None:
                return False
            [hgt,unit] = match.groups()
            hgt = int(hgt)
            if unit == 'cm':
                return hgt >= 150 and hgt <= 193
            elif unit == 'in':
                return hgt >= 59 and hgt <= 76
            else:
                raise f"Unexpected value for hgt {hgt} unit {unit}"
        
        validations = {
            'byr': lambda x: int(x) >= 1920 and int(x) <= 2002,
            'iyr': lambda x: int(x) >= 2010 and int(x) <= 2020,
            'eyr': lambda x: int(x) >= 2020 and int(x) <= 2030,
            'hgt': validate_height,
            'hcl': lambda x: re.match("^#[0-9a-f]{6}$",x) is not None,
            'ecl': lambda x: x in 'amb blu brn gry grn hzl oth'.split(' '),
            'pid': lambda x: re.match("^[0-9]{9}$",x) is not None,
        }
        
        for field in passport:
            if field in validations:
                if not validations[field](passport[field]):
                    return False
        return True

    def day1():
        return len([p for p in passports if valid_1(p)])
    
    def day2():
        return len([p for p in passports if valid_2(p)])
    
    return day1(), day2()

print(f"Day 4: {Day4()}")

Day 4: (190, 121)


In [3]:
# https://adventofcode.com/2020/day/5
def Day5():
    def pid(d):
        row_bits = [1 if b=='B' else 0 for b in d[:7]]
        col_bits = [1 if b=='R' else 0 for b in d[7:]]
        row = bits_to_int(row_bits)
        col = bits_to_int(col_bits)
        return row*8 + col
    
    def pids():
        return [pid(d) for d in get_input(5)]
    
    def part1():
        return max(pids())
    
    def part2():
        pids_ = pids()
        for id in sorted(pids_):
            if (id+1) not in pids_:
                return id + 1
            
    return part1(), part2()
    
print(f"Day 5: {Day5()}")

Day 5: (976, 685)


In [4]:
def Day6():
    data = get_input(6)
    
    def part1():
        groups = []
        group = set()
        for line in data:
            if line == '':
                groups.append(group)
                group = set()
            else:
                for char in line:
                    group.add(char)
            
        if group is not None:
            groups.append(group)
            
        return sum([len(g) for g in groups])
            
    def part2():
        groups = []
        group = {'len': 0}
        for line in data:
            if line == '':
                groups.append(group)
                group = {'len':0}
            else:
                group['len'] += 1
                for char in line:
                    if char not in group:
                        group[char] = 1
                    else:
                        group[char] += 1
        if group is not None:
            groups.append(group)
        
        count = 0
        for group in groups:
            g_len = group['len']
            del group['len']
            count += len([k for k in group if group[k] == g_len])
        return count
            
    
    return part1(), part2()

print(f"Day 6: {Day6()}")

Day 6: (6430, 3125)


In [5]:
def Day7(data=get_input(7)):
    def parse(l):
        l = l[:-1] # drop the '.'

        import re
        container_re = r"^(.*) bags$"
        item_re = r"^(\d+) (.*) bags?$"

        container, items = l.split(' contain ')
        container = re.match(container_re, container).groups()[0]
        items_dict = {}

        if items == 'no other bags':
            pass
        else:
            items = items.split(', ')
            for i in items:
                count,name = re.match(item_re, i).groups()
                count = int(count)
                items_dict[name] = count
        return container,items_dict

    
    def part1():
        nodes = {}
        for line in data:
            container,items = parse(line)
            if container not in nodes:
                nodes[container] = set()
            for item in items:
                if not item in nodes:
                    nodes[item] = set()
                nodes[item].add(container)
        seen = set()
        check = nodes['shiny gold']
        while len(check) > 0:
            cur = check.pop()
            if cur not in seen:
                seen.add(cur)
                check |= nodes[cur]
                
        return len(seen)
            
    def part2():
        
        nodes = {}
        for line in data:
            container,items = parse(line)
            assert(container not in nodes)
            nodes[container] = items
        
        def cost(node):
            return sum([count + count*cost(child) for child,count in nodes[node].items()])
        
        return cost('shiny gold')
                    
    return part1(),part2()

ex1 = """light red bags contain 1 bright white bag, 2 muted yellow bags.
dark orange bags contain 3 bright white bags, 4 muted yellow bags.
bright white bags contain 1 shiny gold bag.
muted yellow bags contain 2 shiny gold bags, 9 faded blue bags.
shiny gold bags contain 1 dark olive bag, 2 vibrant plum bags.
dark olive bags contain 3 faded blue bags, 4 dotted black bags.
vibrant plum bags contain 5 faded blue bags, 6 dotted black bags.
faded blue bags contain no other bags.
dotted black bags contain no other bags.""".split("\n")
ex2 = """shiny gold bags contain 2 dark red bags.
dark red bags contain 2 dark orange bags.
dark orange bags contain 2 dark yellow bags.
dark yellow bags contain 2 dark green bags.
dark green bags contain 2 dark blue bags.
dark blue bags contain 2 dark violet bags.
dark violet bags contain no other bags.""".split("\n")

assert(Day7(ex1) == (4,32))
assert(Day7(ex2)[1] == 126)

print(f"Day 7: {Day7()}")

Day 7: (372, 8015)


In [6]:
def Day8(data = get_input(8)):
    def parse(ins):
        op,offset = ins.split(' ')
        offset = int(offset)
        return (op,offset)
        
    
    def evaluate(instructions):
        acc,pc = (0,0)
        states = []
        seen = []
        delta = 1
        
        def process(op, offset):
            nonlocal acc
            nonlocal pc

            if op == 'acc':
                acc += offset
                pc += delta
            elif op == 'nop':
                pc += delta
            elif op == 'jmp':
                pc += delta*offset
            else:
                assert(False)
        
        while True:
            if pc in seen:
                break
            states.append( (acc, pc) )
            seen.append(pc)
            if pc >= len(instructions) or pc < 0:
                break
            ins = instructions[pc]
            process(*ins)

        return states
            
    
    def part1():
        instructions = [parse(ins) for ins in data]
        states = evaluate(instructions)
        return states[-1][0]
    
    def part2():
        instructions = [parse(ins) for ins in data]
        instruction_copies = []
        for idx,(op,offset) in enumerate(instructions):
            if op in ['jmp','nop']:
                copy = instructions.copy()
                other = 'jmp' if op == 'nop' else 'nop'
                copy[idx] = (other, offset)
                instruction_copies.append(copy)
        for copy in instruction_copies:
            states = evaluate(copy)
            (acc,pc) = states[-1]
            if pc == len(copy): # completed
                return acc

        
    
    return part1(),part2()

print(f"Day 8: {Day8()}")

Day 8: (1928, 1319)


In [7]:
def Day9(size=25):
    data = [int(i) for i in get_input(9)]

    def is_error(arr, d):
        for i in range(len(arr)):
            for j in range(i+1, len(arr)):
                if arr[i] + arr[j] == d:
                    return False
        return True
    
    def part1():
        queue = []
        for d in data:
            if len(queue) < size:
                queue.append(d)
            else:
                if is_error(queue,d):
                    return d
                queue.append(d)
                queue = queue[-size:]
        assert(False)
            
    
    def sum_from(i, data, target):
        start = i
        acc = 0
        while i < len(data):
            acc += data[i]
            if acc == target:
                return data[start:i+1]
            if acc > target:
                return None
            i += 1
        return None
    
    def part2():
        target = part1()
        for i in range(len(data)):
            found = sum_from(i, data, target)
            if found is not None:
                return min(found) + max(found)

    return part1(),part2()

print(f"Day 9: {Day9()}")

Day 9: (27911108, 4023754)


In [8]:
def Day10(data=get_input(10)):
    from collections import defaultdict
    import time
    
    data = [int(d) for d in data]
    _max = max(data)
    deltas = defaultdict(lambda: 0)
    data = sorted([0] + data + [_max + 3])

    def part1(data):
        acc = 0
        for i in range(1, len(data)):
            delta = data[i] - data[i-1]
            deltas[delta] += 1
        return deltas[1] * deltas[3]
    
    def part2(data):
        start = time.perf_counter()
        def count_tail(arr, i=0, memo=defaultdict(lambda:0)):
            if memo[i]:
                return memo[i]
            if i >= len(arr) - 1:
                return 1
            else:
                count = 0
                if i + 3 < len(arr) and arr[i+3] - arr[i] <= 3:
                    count += count_tail(arr, i + 3, memo)
                if i + 2 < len(arr) and arr[i+2] - arr[i] <= 3:
                    count += count_tail(arr, i + 2, memo)
                if i + 1 < len(arr):
                    count += count_tail(arr, i + 1, memo)
                memo[i] = count
                return count
            
        result = count_tail(data)
        print(f"Part 2 Took {time.perf_counter() - start}")
        return result
        
    
    return part1(data), part2(data)

ex1 = [16, 10, 15, 5, 1, 11, 7, 19, 6, 12, 4]
ex2 = [28, 33, 18, 42, 31, 14, 46, 20, 48, 47, 24, 23, 49, 45, 19, 38, 39, 11, 1, 32, 25, 35, 8, 17, 7, 9, 4, 2, 34, 10, 3]

print(f"Day 10: {Day10()}")

Part 2 Took 0.00017533800000002486
Day 10: (1690, 5289227976704)


In [9]:
def Day11(data = get_input(11)):
    from collections import defaultdict

    FLOOR = 0
    EMPTY = 1
    OCC   = 2
    OOB   = -1
    STR_TO_GRID = {
        'L': EMPTY,
        '.': FLOOR,
        '#': OCC        
    }
    GRID_TO_STR = {
        EMPTY: 'L',
        FLOOR: '.',
        OCC:   '#'
    }
    
    def to_grid(data):
        grid = defaultdict(lambda: OOB)
        for y,row in enumerate(data):
            for x,col in enumerate(row):
                grid[(x,y)] = STR_TO_GRID[col]
        return grid
    
    def grid_to_str(grid):
        xmax = max([point[0] for point in grid.keys()])
        ymax = max([point[1] for point in grid.keys()])
        rows = []
        for y in range(ymax):
            col = []
            for x in range(xmax):
                col.append(GRID_TO_STR[grid[(x,y)]])
            rows.append(''.join([str(c) for c in col]))
        return "\n".join(rows)
    
    def neighbors(grid, point):
        (x,y) = point
        dxs = [-1, 0, 1]
        dys = [-1, 0, 1]
        ns = []
        for dx in dxs:
            for dy in dys:
                if dx == 0 and dy == 0:
                    continue
                ns.append(grid[(x+dx,y+dy)])
        return [n for n in ns if n != OOB]
    
    def line_of_sight_neighbors(grid, point):
        (x,y) = point
        dxs = [-1, 0, 1]
        dys = [-1, 0, 1]
        ns = []
        for dx in dxs:
            for dy in dys:
                if dx == 0 and dy == 0:
                    continue
                cur_x = x
                cur_y = y
                while True:
                    cur_x += dx
                    cur_y += dy
                    seat = grid[(cur_x,cur_y)]
                    if seat == OOB:
                        break
                    if seat == FLOOR:
                        continue
                    ns.append(seat)
                    break
        return ns
    
    def tick(grid):
        new_grid = defaultdict(lambda:OOB)
        changes = 0
        for point in list(grid.keys()):
            (x,y) = point
            center = grid[point]
            if center == FLOOR:
                new_grid[point] = FLOOR
            else:
                ns = neighbors(grid, point)
                occ_count = len([n for n in ns if n == OCC])
                
                if center == EMPTY:
                    if occ_count == 0:
                        changes += 1
                        new_grid[point] = OCC
                    else:
                        new_grid[point] = EMPTY
                elif center == OCC:
                    if occ_count >= 4:
                        changes += 1
                        new_grid[point] = EMPTY
                    else:
                        new_grid[point] = OCC
                else:
                    assert(False)
        return (new_grid, changes)
    
    def line_of_sight_tick(grid):
        new_grid = defaultdict(lambda:OOB)
        changes = 0
        for point in list(grid.keys()):
            (x,y) = point
            center = grid[point]
            if center == FLOOR:
                new_grid[point] = FLOOR
            else:
                ns = line_of_sight_neighbors(grid, point)
                occ_count = len([n for n in ns if n == OCC])
                
                if center == EMPTY:
                    if occ_count == 0:
                        changes += 1
                        new_grid[point] = OCC
                    else:
                        new_grid[point] = EMPTY
                elif center == OCC:
                    if occ_count >= 5:
                        changes += 1
                        new_grid[point] = EMPTY
                    else:
                        new_grid[point] = OCC
                else:
                    assert(False)
        return (new_grid, changes)
    
    def part1(data):
        grid = to_grid(data)

        while True:
            (grid,changes) = tick(grid)
            if changes == 0:
                break
        return len([seat for seat in grid if grid[seat] == OCC])
    
    def part2(data):
        grid = to_grid(data)
        while True:
            (grid,changes) = line_of_sight_tick(grid)
            if changes == 0:
                break
        return len([seat for seat in grid if grid[seat] == OCC])
                        
    return part1(data),part2(data)

ex = """L.LL.LL.LL
LLLLLLL.LL
L.L.L..L..
LLLL.LL.LL
L.LL.LL.LL
L.LLLLL.LL
..L.L.....
LLLLLLLLLL
L.LLLLLL.L
L.LLLLL.LL""".split("\n")

print(f"Day 11: {Day11()}")

Day 11: (2476, 2257)


In [10]:
def Day12(data=get_input(12)):
    def day1(data):
        x = 0
        y = 0
        cur_dir = (1,0) # East
        
        W = (-1,0)
        E = (1,0)
        N = (0,-1)
        S = (0,1)
        dir_vectors = {
            'W': W,
            'N': N,
            'E': E,
            'S': S
        }

        def turn(cur_dir, angle):
            assert(angle % 90 == 0)
            next_dir = cur_dir
            while angle > 0: # turning right
                next_dir = turn_90(next_dir, 'R')
                angle -= 90
            while angle < 0: # turning left
                next_dir = turn_90(next_dir, 'L')
                angle += 90
            return next_dir
            
        def turn_90(cur_dir, l_or_r):
            r_dirs = [
                E, S, W, N, E
            ]
            l_dirs = [
                E, N, W, S, E
            ]
            if l_or_r == 'L':
                next_dir = l_dirs[l_dirs.index(cur_dir)+1]
            else:
                next_dir = r_dirs[r_dirs.index(cur_dir)+1]
            return next_dir

        for l in data:
            ins,amt = l[:1],int(l[1:])
            if ins == 'L' or ins == 'R':
                if ins == 'L':
                    amt *= -1
                cur_dir = turn(cur_dir, amt)
            elif ins == 'F':
                x += cur_dir[0] * amt
                y += cur_dir[1] * amt
            else:
                vector = dir_vectors[ins]
                x += vector[0] * amt
                y += vector[1] * amt

        return manhattan_distance( (0,0), (x,y) )
    
    def day2(data):
        def rotate(vec, angle):
            assert(angle % 90 == 0)
            while angle > 0:
                vec = rotate_90(vec, clockwise=True)
                angle -= 90
            while angle < 0:
                vec = rotate_90(vec, clockwise=False)
                angle += 90
            return vec
        
        def rotate_90(vec, clockwise=True):
            if clockwise:
                return (-1*vec[1],vec[0])
            else:
                return (vec[1], -1*vec[0])
            
        way = (10,-1)
        W = (-1,0)
        E = (1,0)
        N = (0,-1)
        S = (0,1)
        dir_vectors = {
            'W': W,
            'N': N,
            'E': E,
            'S': S
        }
        cur = (0,0)
        for l in data:
            ins,amt = l[:1],int(l[1:])
            if ins == 'L' or ins == 'R':
                if ins == 'L':
                    amt *= -1
                way = rotate(way, amt)
            elif ins == 'F':
                cur = (
                    cur[0] + way[0]*amt,
                    cur[1] + way[1]*amt
                )
            else: # NEWS
                vector = dir_vectors[ins]
                way = (
                    way[0] + vector[0]*amt,
                    way[1] + vector[1]*amt
                )
            
        return manhattan_distance( (0,0), cur )
        
        
    return day1(data),day2(data)

ex = """F10
N3
F7
R90
F11""".split("\n")

print(f"Day 12: {Day12()}")

Day 12: (420, 42073)


In [11]:
def Day13(data=get_input(13)):
    def day1(data):
        stamp = int(data[0])
        buses = [int(d) for d in data[1].split(',') if d != 'x']
        bus_mins = {bus:0 for bus in buses}
        for bus in buses:
            acc = 0
            while acc < stamp:
                acc += bus
            bus_mins[bus] = acc - stamp
        minwait = min(bus_mins.values())
        bus = [bus for bus in buses if bus_mins[bus] == minwait][0]
        return bus*minwait
    
    
    def day2(data):
        equivs = [(idx,int(d)) for idx,d in enumerate(data[1].split(',')) if d != 'x']

        def solve_congruence(a1,n1,a2,n2):
            """
            https://en.wikipedia.org/wiki/Chinese_remainder_theorem#Using_the_existence_construction
            """
            (bez_n1, bez_n2) = bezout_coeff(n1,n2)

            # bezout's identity:
            # bez_n1 * n1 + bez_n2 * n2 = 1

            # existence formula
            # a_2 * (bez_n1 * n1) + a_1 * (bez_n2 * n2)

            # congruence solution
            next_a = a2 * (bez_n1 * n1) + a1 * (bez_n2 * n2)
            next_n = n1*n2

            return (next_a, next_n)
        
        (a1,n1) = equivs.pop()

        while len(equivs):
            (a2,n2) = equivs.pop()
            (a1,n1) = solve_congruence(a1,n1,a2,n2)
            
        # smallest positive integer
        return n1%(a1%n1)

    return day1(data),day2(data)

ex = """100
7,13,x,x,59,x,31,19""".split("\n")

print(f"Day 13: {Day13()}")

Day 13: (102, 327300950120029)


In [12]:
def Day16(data=get_input(16)):
    from collections import defaultdict
    
    def parse(data):
        (ranges, ticket, nearby) = split_list(data, '')
        ticket = list(map(int, ticket[1].split(',')))
        nearby = [list(map(int, line.split(','))) for line in nearby[1:]]
        
        _ranges = {}
        for line in ranges:
            key = line.split(': ')[0]
            seqs = line.split(': ')[1].split(' or ')
            seqs = [list(map(int, v.split('-'))) for v in seqs]
            _ranges[key] = seqs
        ranges = _ranges
        
        return ranges,ticket,nearby
    
    ranges,your_ticket,nearby = parse(data)
    
    def valid_value(v, ranges):
        for seqs in ranges.values():
            if valid_value_range(v, seqs):
                return True
        return False
    
    def valid_value_range(v, ranges):
        return any([range_[0] <= v <= range_[1] for range_ in ranges])
    
    def day1():
        return sum([d for ticket in nearby for d in ticket if not valid_value(d, ranges)])

    def day2():
        valid_nearby = [ticket for ticket in nearby if all([valid_value(d, ranges) for d in ticket])]
        impossible_fields = defaultdict(set)
        for ticket in valid_nearby:
            for idx,v in enumerate(ticket):
                for field,seqs in ranges.items():
                    if not valid_value_range(v,seqs):
                        impossible_fields[field].add(idx)
        possible_fields = {}
        for key,impossible in impossible_fields.items():
            possible_fields[key] = set(range(len(ranges))).difference(impossible)
        
        certain_fields = {}
        while len(certain_fields) < len(ranges) - 1:
            found_field = None
            found_field_idx = None
            for field,possible in possible_fields.items():
                if len(possible) == 1:
                    found_field = field
                    found_field_idx = list(possible)[0]
                    break

            if found_field is None:
                print(certain_fields, len(certain_fields))
                break
            else:
                certain_fields[found_field] = found_field_idx
                for field in possible_fields:
                    possible_fields[field] -= {found_field_idx}
        
        last_field = set(ranges.keys()) - set(certain_fields.keys())
        assert(len(last_field) == 1)
        last_field_idx = set(range(len(ranges))) - set(certain_fields.values())
        assert(len(last_field_idx) == 1)
        last_field = last_field.pop()
        last_field_idx = last_field_idx.pop()
        certain_fields[last_field] = last_field_idx
        
        target_fields = [key for key in ranges.keys() if key.startswith('departure')]
        
            
        # double-check that every ticket is indeed valid assuming these certain_fields
#         print(certain_fields)
#         for ticket in valid_nearby:
#             for field in certain_fields:
#                 seqs = ranges[field]
#                 idx = certain_fields[field]
#                 val = ticket[idx]
#                 assert(valid_value_range(val, seqs))

        val = 1
#         print(your_ticket)
        for field in target_fields:
#             print(f"field {field}, idx {certain_fields[field]}, val {your_ticket[certain_fields[field]]}")
            val *= your_ticket[certain_fields[field]]
        return val

    return day1(),day2()
        

Day16()

(20091, 2325343130651)

In [13]:
def Day17(data=get_input(17)):
    from collections import defaultdict
    
    ALIVE = '#'
    ACTIVE = ALIVE
    EMPTY = '.'
    
    def neighbors(point, grid, dim=3):
        if dim == 3:
            return neighbors3(point, grid)
        else:
            return neighbors4(point, grid)
        
    def neighbors3(point,grid):
        deltas = [-1, 0, 1]
        deltas = [(dx,dy,dz) for dx in deltas for dy in deltas for dz in deltas if not dx == dy == dz == 0]
        (x,y,z) = point
        return {(x+dx,y+dy,z+dz):grid[(x+dx,y+dy,z+dz)] for (dx,dy,dz) in deltas}
    
    def neighbors4(point,grid):
        deltas = [-1, 0, 1]
        deltas = [(dx,dy,dz,dw) for dx in deltas for dy in deltas for dz in deltas for dw in deltas if not dx == dy == dz == dw == 0]
        (x,y,z,w) = point
        return {(x+dx,y+dy,z+dz,w+dw):grid[(x+dx,y+dy,z+dz,w+dw)] for (dx,dy,dz,dw) in deltas}
    
    
    def tick(grid):
        next_grid = defaultdict(lambda: EMPTY)
        
        (mins,maxes) = grid_limits(grid)
        
        for x in range(mins[0] - 1, maxes[0] + 2):
            for y in range(mins[1] - 1, maxes[1] + 2):
                for z in range(mins[2] - 1, maxes[2] + 2):
                    point = (x,y,z)
                    ns = neighbors(point, grid)
                    alive = len([v for v in ns.values() if v == ALIVE])

                    next_val = None
                    if grid[point] == ALIVE:
                        if alive == 2 or alive == 3:
                            next_val = ALIVE
                        else:
                            next_val = EMPTY
                    else:
                        if alive == 3:
                            next_val = ALIVE
                        else:
                            next_val = EMPTY
                    assert(next_val is not None)
                    
                    next_grid[point] = next_val

        return next_grid
    
    def tick4(grid):
        next_grid = defaultdict(lambda: EMPTY)
        
        (mins,maxes) = grid_limits(grid,dim=4)
        
        for x in range(mins[0] - 1, maxes[0] + 2):
            for y in range(mins[1] - 1, maxes[1] + 2):
                for z in range(mins[2] - 1, maxes[2] + 2):
                    for w in range(mins[3] -1, maxes[3] + 2):
                        point = (x,y,z,w)
                        ns = neighbors4(point, grid)
                        alive = len([v for v in ns.values() if v == ALIVE])

                        next_val = None
                        if grid[point] == ALIVE:
                            if alive == 2 or alive == 3:
                                next_val = ALIVE
                            else:
                                next_val = EMPTY
                        else:
                            if alive == 3:
                                next_val = ALIVE
                            else:
                                next_val = EMPTY
                        assert(next_val is not None)

                        next_grid[point] = next_val

        return next_grid
    
    def grid_limits(grid, dim=3):
        mins =   [min([point[i] for point in grid]) for i in range(dim)]
        maxes =  [max([point[i] for point in grid]) for i in range(dim)]
        return (mins, maxes)
        
    
    def day1():
        grid = defaultdict(lambda: EMPTY)
        
        z = 0
        for y,row in enumerate(data):
            for x,col in enumerate(row):
                grid[(x,y,z)] = col

        for _ in range(6):
            grid = tick(grid)
            
        
        count = len([v for v in grid.values() if v == ACTIVE])
        return count

    def day2():
        grid = defaultdict(lambda: EMPTY)
        
        z = 0
        w = 0
        for y,row in enumerate(data):
            for x,col in enumerate(row):
                grid[(x,y,z,w)] = col

        for _ in range(6):
            grid = tick4(grid)
            
        
        count = len([v for v in grid.values() if v == ACTIVE])
        return count
        
    
    return day1(),day2()

ex = """.#.
..#
###""".split('\n')

print(f"Day 17: {Day17()}")

Day 17: (263, 1680)


In [14]:
def Day18(data=get_input(18)):
    import operator
    
    def parse_term(tokens):
        assert(len(tokens) > 0)
        token = tokens[0]
        return ('TERM',int(token)),tokens[1:]

    def parse_op(tokens):
        assert(len(tokens) > 0)
        token = tokens[0]
        op = None
        assert token in ['*','+'], f"Expected op, got {token}"
        return ('OP',token),tokens[1:]

    def tokenize(s):
        return list(s.replace(' ', ''))

    def parse_term_or_expr(tokens):
        token = tokens[0]
        if token == '(':
            return parse_expr(tokens[1:])
        else:
            return parse_term(tokens)

    def parse_expr(tokens):
        expr = []

        node,tokens = parse_term_or_expr(tokens)
        expr.append(node)

        if len(tokens) == 0:
            return ('EXPR',expr)
        if tokens[0] == ')':
            return ('EXPR',expr),tokens[1:]

        op,tokens = parse_op(tokens)
        expr.append(op)

        while len(tokens) > 0:
            node,tokens = parse_term_or_expr(tokens)
            expr.append(node)

            if len(tokens) > 0:
                if tokens[0] == ')':
                    return ('EXPR',expr),tokens[1:]
                elif tokens[0] == '(':
                    sub_expr,tokens = parse_expr(tokens[1:])
                    expr.append(sub_expr)
                else:
                    op,tokens = parse_op(tokens)
                    expr.append(op)
        return ('EXPR',expr)

    def eval_node(node):
        if node[0] == 'TERM':
            return node[1]
        else:
            assert node[0] == 'EXPR'
            return eval_expr(node)

    def eval_expr(expr):
        assert expr[0] == 'EXPR', f"Unexpected expr {expr}"
        nodes = expr[1]
        val = None
        op = None

        assert len(nodes) % 2 == 1
        while len(nodes) > 0:
            node,nodes = nodes[0],nodes[1:]

            if node[0] == 'OP':
                assert val is not None
                assert len(nodes) > 0

                op = operator.add if node[1] == '+' else operator.mul
                node,nodes = nodes[0],nodes[1:]
                val = op(val, eval_node(node))
            else:
                val = eval_node(node)

        return val
    
    def eval_node2(node):
        if node[0] == 'TERM':
            return node[1]
        else:
            assert node[0] == 'EXPR'
            return eval_expr2(node)
    
    def eval_expr2(expr):
        assert expr[0] == 'EXPR', f"Unexpected expr {expr}"
        nodes = expr[1]
        val = None
        op = None
        
        assert len(nodes) % 2 == 1
        
        while findindex(nodes, lambda n: n[1] == '+') is not None:
            plus_idx = findindex(nodes, lambda n: n[1] == '+')
#             print(f"pidx {plus_idx}, before node {nodes[plus_idx-1]}, after node {nodes[plus_idx+1]}")
            v1 = eval_node2(nodes[plus_idx-1])
            v2 = eval_node2(nodes[plus_idx+1])
#             print(f"adding {v1} + {v2} (plus index: {plus_idx})")
#             print(f"old nodes: {nodes}")
            nodes = nodes[0:plus_idx - 1] + [('TERM',v1+v2)] + nodes[plus_idx + 2:]
#             print(f"new nodes: {nodes}")

        while len(nodes) > 0:
            node,nodes = nodes[0],nodes[1:]

            if node[0] == 'OP':
                assert val is not None
                assert len(nodes) > 0

                op = operator.add if node[1] == '+' else operator.mul
                node,nodes = nodes[0],nodes[1:]
                val = op(val, eval_node2(node))
            else:
                val = eval_node2(node)

        return val

    def eval_str(s):
        return eval_expr(parse_expr(tokenize(s)))
    
    def part1():
        return sum([eval_str(line) for line in data])

    def part2():
        return sum([eval_expr2(parse_expr(tokenize(line))) for line in data])
    
    assert eval_expr2(parse_expr(tokenize('1 + 2 * 3 + 4 * 5 + 6'))) == 231
    assert eval_expr2(parse_expr(tokenize('2 * 3 + (4 * 5)'))) == 46
    assert eval_expr2(parse_expr(tokenize('5+(8 * 3 + 9 + 3 * 4 * 3)'))) == 1445
    assert eval_expr2(parse_expr(tokenize("5 * 9 * (7 * 3 * 3 + 9 * 3 + (8 + 6 * 4))"))) == 669060
    assert eval_expr2(parse_expr(tokenize("((2 + 4 * 9) * (6 + 9 * 8 + 6) + 6) + 2 + 4 * 2"))) == 23340
    assert eval_expr2(parse_expr(tokenize("1 + (2 * 3) + (4 * (5 + 6))"))) == 51
    
    return part1(),part2()
    
print(f"Day 18: {Day18()}")

Day 18: (24650385570008, 158183007916215)


In [15]:
def Day19(data=get_input(19)):
    def parse_rule(s):
        num,rule = s.split(': ')
        num = int(num)
        raw_options = rule.split(' | ')
        options = []
        for raw_option in raw_options:
            if '"' in raw_option:
                options.append(('CHAR',raw_option.replace('"','')))
            else:
                nums = list(map(int, raw_option.split(' ')))
                options.append(('SEQ',nums))
        if len(options) > 1:
            rule = ('OR',options)
        else:
            rule = options[0]
        return (num,rule)

    def parse_rules(rules):
        out = {}
        for line in rules:
            num,rule = parse_rule(line)
            out[num] = rule
        return out
    
    def to_re(rule, ruleset):
        if rule[0] == 'CHAR':
            return rule[1]
        elif rule[0] == 'OR':
            return f"({to_re(rule[1][0], ruleset)}|{to_re(rule[1][1], ruleset)})"
        else:
            out = ""
            rules = [ruleset[num] for num in rule[1]]
            for r in rules:
                out += to_re(r,ruleset)
            return out

    def to_re2(rule, ruleset):
        if rule == ruleset[8]:
            return f"({to_re2(ruleset[42], ruleset)})+"
        if rule == ruleset[11]:
            r42 = to_re2(ruleset[42], ruleset)
            r31 = to_re2(ruleset[31], ruleset)
            out = []
            for i in range(1,5):
                out.append("((" + r42 + "){" + str(i) + "}" + "(" + r31 + "){" + str(i) + "}" + ")")
            out = "|".join(out)
            r11 = f"({out})"
            return r11
        
        if rule[0] == 'CHAR':
            return rule[1]
        elif rule[0] == 'OR':
            return f"({to_re2(rule[1][0], ruleset)}|{to_re2(rule[1][1], ruleset)})"
        else:
            out = ""
            rules = [ruleset[num] for num in rule[1]]
            for r in rules:
                out += to_re2(r,ruleset)
                
            return out
        
    def parse_data(data):
        rules,strings = split_list(data,'')
        ruleset = parse_rules(rules)
        return ruleset,strings
    
    
    def validate_rule(s, rule, ruleset):
        import re
        _re = f"^{to_re(rule, ruleset)}$"
        return re.match(_re, s) is not None

    def validate_rule2(s, rule, ruleset):
        import re
        _re = f"^{to_re2(rule, ruleset)}$"
        return re.match(_re, s) is not None

    
    def part1(data):
        ruleset,strings = parse_data(data)
        return len([s for s in strings if validate_rule(s, ruleset[0], ruleset)])
    
    def part2(data):
        ruleset,strings = parse_data(data)
        return len([s for s in strings if validate_rule2(s, ruleset[0], ruleset)])        

    return part1(data),part2(data)

ex = """0: 4 1 5
1: 2 3 | 3 2
2: 4 4 | 5 5
3: 4 5 | 5 4
4: "a"
5: "b"

ababbb
bababa
abbbab
aaabbb
aaaabbb""".split("\n")

ex2 = """42: 9 14 | 10 1
9: 14 27 | 1 26
10: 23 14 | 28 1
1: "a"
11: 42 31
5: 1 14 | 15 1
19: 14 1 | 14 14
12: 24 14 | 19 1
16: 15 1 | 14 14
31: 14 17 | 1 13
6: 14 14 | 1 14
2: 1 24 | 14 4
0: 8 11
13: 14 3 | 1 12
15: 1 | 14
17: 14 2 | 1 7
23: 25 1 | 22 14
28: 16 1
4: 1 1
20: 14 14 | 1 15
3: 5 14 | 16 1
27: 1 6 | 14 18
14: "b"
21: 14 1 | 1 14
25: 1 1 | 1 14
22: 14 14
8: 42
26: 14 22 | 1 20
18: 15 15
7: 14 5 | 1 21
24: 14 1

abbbbbabbbaaaababbaabbbbabababbbabbbbbbabaaaa
bbabbbbaabaabba
babbbbaabbbbbabbbbbbaabaaabaaa
aaabbbbbbaaaabaababaabababbabaaabbababababaaa
bbbbbbbaaaabbbbaaabbabaaa
bbbababbbbaaaaaaaabbababaaababaabab
ababaaaaaabaaab
ababaaaaabbbaba
baabbaaaabbaaaababbaababb
abbbbabbbbaaaababbbbbbaaaababb
aaaaabbaabaaaaababaa
aaaabbaaaabbaaa
aaaabbaabbaaaaaaabbbabbbaaabbaabaaa
babaaabbbaaabaababbaabababaaab
aabbbbbaabbbaaaaaabbbbbababaaaaabbaaabba""".split("\n")

print(f"Day 19: {Day19()}")

Day 19: (203, 304)


In [16]:
def Day20(data = get_input(20)):
    def get_tiles(data):
        tiles = []
        for tile_data in split_list(data,''):
            header,data = tile_data[0],tile_data[1:]
            num = int(header[-5:-1])
            tiles.append((num, data))
        return tiles
    
    def part1(data):
        tiles = get_tiles(data)

        def borders(tile):
            rows = tile[1]
            bs = []
            bs.append(rows[0])
            bs.append(rows[-1])
            bs.append(''.join([row[0] for row in rows])) #col 0
            bs.append(''.join([row[-1] for row in rows])) #last col
            flips = []
            for b in bs:
                flips.append(b[::-1])
            return bs + flips

        val = 1
        for tile in tiles:
            count = 0
            bs = borders(tile)
            for b in bs:
                for other_tile in tiles:
                    if tile == other_tile:
                        continue
                    other_bs = borders(other_tile)
                    if b in other_bs:
                        count += 1
            if count == 4:
                val *= tile[0]
        return val
    
    def print_tile(tile):
        out = "\n".join(tile[1])
        print(out)
    
    
    def part2(data):
        from collections import defaultdict
        import math
        OOB = -1
        EMPTY = 0
        DIRS = ['N','E','S','W']
        tiles = get_tiles(data)
        SEA_MONSTER_PATTERNS = [
            [(18, 0), (0, 1), (5, 1), (6, 1), (11, 1), (12, 1), (17, 1), (18, 1), (19, 1), (1, 2), (4, 2), (7, 2), (10, 2), (13, 2), (16, 2)],
            [(1, 0), (0, 1), (1, 1), (2, 1), (7, 1), (8, 1), (13, 1), (14, 1), (19, 1), (3, 2), (6, 2), (9, 2), (12, 2), (15, 2), (18, 2)],
            [(1, 0), (2, 1), (2, 4), (1, 5), (1, 6), (2, 7), (2, 10), (1, 11), (1, 12), (2, 13), (2, 16), (1, 17), (0, 18), (1, 18), (1, 19)],
            [(1, 0), (0, 1), (0, 4), (1, 5), (1, 6), (0, 7), (0, 10), (1, 11), (1, 12), (0, 13), (0, 16), (1, 17), (1, 18), (2, 18), (1, 19)]
        ]
                            
            
        def make_grid(size):
            grid = defaultdict(lambda: OOB)
            for x in range(size):
                for y in range(size):
                    grid[(x,y)] = EMPTY
            return grid

        grid = make_grid(int(math.sqrt(len(tiles))))

        def rot_tile(tile, cw_turns=1):
            if cw_turns == 0:
                return tile
            num,rows = tile
            cols = [''.join(reversed([row[i] for row in rows])) for i in range(len(rows))]
            new_rows = cols
            new_tile = (num, new_rows)
            return rot_tile(new_tile, cw_turns - 1)
    
        def flip_tile(tile):
            num,rows = tile
            return(num, [row[::-1] for row in rows])

        
        def all_borders(tile):
            borders = [border(tile,face) for face in DIRS]
            reversed_borders = [b[::-1] for b in borders]
            return borders + reversed_borders
        
        def border(tile, face):
            num,data = tile
            if face == 'N':
                return data[0]
            elif face == 'E':
                return ''.join([row[-1] for row in data])
            elif face == 'S':
                return data[-1]
            elif face == 'W':
                return ''.join([row[0] for row in data])
        
        def find_corners(tiles):
            corner_tiles = []

            for tile in tiles:
                count = 0
                bs = all_borders(tile)
                for b in bs:
                    for other_tile in tiles:
                        if tile == other_tile:
                            continue
                        other_bs = all_borders(other_tile)
                        if b in other_bs:
                            count += 1
                if count == 4:
                    corner_tiles.append(tile)

            return corner_tiles
        
        def tile_variants(tile):
            variants = []
            
            for turn in range(4):
                variants.append(rot_tile(tile,turn))
            tile = flip_tile(tile)
            for turn in range(4):
                variants.append(rot_tile(tile,turn))
            return variants
                
        
        def orient_tile(tile, border_faces):
            assert all(f in DIRS for f in border_faces)
            variants = tile_variants(tile)
            for v in variants:
                if all(border(v,face) == b for face,b in border_faces.items()):
                    return v
                
         
        def find_oriented_corner_tile(tiles):
            corner = find_corners(tiles)[0]
            corner_bs = all_borders(corner)
            matching_corner_bs = []
            for tile in tiles:
                if tile == corner:
                    continue
                bs = all_borders(tile)
                for b in bs:
                    if b in corner_bs:
                        matching_corner_bs.append(b)
            assert(len(matching_corner_bs) == 4)

            # remove reversed first border
            matching_corner_bs.remove(matching_corner_bs[0][::-1])

            # remove reversed first border
            matching_corner_bs.remove(matching_corner_bs[1][::-1])

            assert(len(matching_corner_bs) == 2)

            border_faces = {'E': matching_corner_bs[0][::-1], 'S': matching_corner_bs[1]}
            oriented_tile = orient_tile(corner, border_faces)
            return oriented_tile
        
        def find_matching_tile(tiles, border_faces):
            matched = []
            for tile in tiles:
                oriented = orient_tile(tile,border_faces)
                if oriented is not None:
                    matched.append(oriented)
            assert len(matched) == 1, f"Could not find a match for {border_faces}, got: {len(matched)}"
            return matched[0]

        DIR_DELTAS = {'N': (0,-1), 'E': (1,0), 'S': (0,1), 'W': (-1,0)}
        OPP_DIRS = {'N':'S', 'E':'W', 'W':'E', 'S':'N'}


        # returns the points that are empty around this point
        def empty_neighbors(grid, point):
            (x,y) = point
            neighbors = []
            for (dx,dy) in DIR_DELTAS.values():
                neighbor = (x+dx,y+dy)
                if grid[neighbor] == EMPTY:
                    neighbors.append(neighbor)
            return neighbors
        
        def occupied_neighbors(grid, point):
            (x,y) = point
            neighbors = []
            for (dx,dy) in DIR_DELTAS.values():
                neighbor = (x+dx,y+dy)
                if grid[neighbor] != EMPTY and grid[neighbor] != OOB:
                    neighbors.append(neighbor)
            return neighbors
        
        def find_border_faces(grid, point):
            assert(grid[point] == EMPTY)
            (x,y) = point
            border_faces = {}
            for d,deltas in DIR_DELTAS.items():
                neighbor = grid[(x+deltas[0],y+deltas[1])]
                if neighbor != EMPTY and neighbor != OOB:
                    border_faces[d] = border(neighbor, OPP_DIRS[d])
            assert(len(border_faces) > 0)
            return border_faces
        
        def remove_tile(tiles,tile):
            return [t for t in tiles if t[0] != tile[0]]
        
        def remove_borders(grid):
            new_grid = {}
            max_x = max(x for (x,y) in grid)
            max_y = max(y for (x,y) in grid)
            assert(max_x == max_y)
            
            for (point,tile) in grid.items():
                (x,y) = point
                (num,rows) = tile

                # remove 1st/last row
                rows = rows[1:-1]
                rows = [row[1:-1] for row in rows]
                tile = (num,rows)
                new_grid[point] = tile
            return new_grid

        
        def tile_grid_to_char_grid(grid):
            some_tile = grid[(0,0)]
            (_, tile_rows) = some_tile
            assert(len(tile_rows) == len(tile_rows[0]))
            size = len(tile_rows)

            char_grid = {}
            for tile_point,tile in grid.items():
                (tile_x,tile_y) = tile_point
                assert(tile != OOB and tile != EMPTY)
                (num, rows) = tile
                base_ch_x = tile_x * size
                base_ch_y = tile_y * size
                for row_idx,row in enumerate(rows):
                    char_y = base_ch_y + row_idx
                    for ch_idx,ch in enumerate(row):
                        char_x = base_ch_x + ch_idx
                        ch_point = (char_x, char_y)
                        char_grid[ch_point] = ch

            return char_grid
            
        def check_pattern(grid, point, pattern):
            (x,y) = point
            for (dx,dy) in pattern:
                check_point = (x+dx,y+dy)
                if check_point not in grid:
                    return False
                elif grid[check_point] != '#':
                    return False
            return True
            
        def print_char_grid(grid):
            max_x = max(point[0] for point in grid)
            max_y = max(point[1] for point in grid)
            
            for y in range(max_y+1):
                line = ""
                for x in range(max_x+1):
                    line += grid[(x,y)]
                print(line)
                
        
        corner = find_oriented_corner_tile(tiles)
        tiles = remove_tile(tiles,corner)
        
        grid[(0,0)] = corner
        seen = set()
        empty_spaces = empty_neighbors(grid,(0,0))
        
        while len(tiles) > 0:
            if len(tiles) == 0:
                break
            if len(empty_spaces) == 0:
                break
            
            neighbor_counts = [len(occupied_neighbors(grid, e_s)) for e_s in empty_spaces]
            max_empty_points = [e_s for e_s in empty_spaces if len(occupied_neighbors(grid,e_s)) == max(neighbor_counts)]
            assert(len(max_empty_points) > 0)
            empty_point = max_empty_points[0]
            
            if empty_point in seen:
                empty_spaces.remove(empty_point)
                continue
            seen.add(empty_point)
            border_faces = find_border_faces(grid, empty_point)
            tile = find_matching_tile(tiles, border_faces)
            grid[empty_point] = tile
            tiles = remove_tile(tiles, tile)
            for e_n in empty_neighbors(grid, empty_point):
                if e_n not in empty_spaces and e_n not in seen:
                    empty_spaces.append(e_n)

        grid = {point:tile for point,tile in grid.items() if tile != EMPTY and tile != OOB}
        grid = remove_borders(grid)
        ch_grid = tile_grid_to_char_grid(grid)
        
        found_count = 0
        for pidx,pattern in enumerate(SEA_MONSTER_PATTERNS):
            for point,char in ch_grid.items():
                if check_pattern(ch_grid, point, pattern):
                    found_count += 1
        return len([v for v in ch_grid.values() if v == '#']) - found_count * len(SEA_MONSTER_PATTERNS[0])
    return part1(data),part2(data)
                
ex = """Tile 2311:
..##.#..#.
##..#.....
#...##..#.
####.#...#
##.##.###.
##...#.###
.#.#.#..##
..#....#..
###...#.#.
..###..###

Tile 1951:
#.##...##.
#.####...#
.....#..##
#...######
.##.#....#
.###.#####
###.##.##.
.###....#.
..#.#..#.#
#...##.#..

Tile 1171:
####...##.
#..##.#..#
##.#..#.#.
.###.####.
..###.####
.##....##.
.#...####.
#.##.####.
####..#...
.....##...

Tile 1427:
###.##.#..
.#..#.##..
.#.##.#..#
#.#.#.##.#
....#...##
...##..##.
...#.#####
.#.####.#.
..#..###.#
..##.#..#.

Tile 1489:
##.#.#....
..##...#..
.##..##...
..#...#...
#####...#.
#..#.#.#.#
...#.#.#..
##.#...##.
..##.##.##
###.##.#..

Tile 2473:
#....####.
#..#.##...
#.##..#...
######.#.#
.#...#.#.#
.#########
.###.#..#.
########.#
##...##.#.
..###.#.#.

Tile 2971:
..#.#....#
#...###...
#.#.###...
##.##..#..
.#####..##
.#..####.#
#..#.#..#.
..####.###
..#.#.###.
...#.#.#.#

Tile 2729:
...#.#.#.#
####.#....
..#.#.....
....#..#.#
.##..##.#.
.#.####...
####.#.#..
##.####...
##..#.##..
#.##...##.

Tile 3079:
#.#.#####.
.#..######
..#.......
######....
####.#..#.
.#...#.##.
#.#####.##
..#.###...
..#.......
..#.###...""".split("\n")

print(f"Day 20: {Day20()}")

Day 20: (174206308298779, 2409)


In [17]:
def Day21(data=get_input(21)):
    from collections import defaultdict

    def part1(p2=False):
        allergens = {}
        recipes = []
        for line in data:
            ingredients, _allergens = line.split(' (contains')
            ingredients = ingredients.split(' ')
            recipes.append(ingredients)
            _allergens = map(lambda x:x.strip(), _allergens[:-1].split(', '))
            for a in _allergens:
                if a in allergens:
                    allergens[a] = allergens[a].intersection(ingredients)
                else:
                    allergens[a] = set(ingredients)

        certain = {}
        while len(allergens) > 0:
            a,i = sorted(allergens.items(), key=lambda x:len(x[1]))[0]
            if len(i) == 1:
                certain[a] = i.pop()
            allergens = {a: i - set(certain.values()) for a,i in allergens.items() if a not in certain}
        
        known = certain.values()
        
        from collections import Counter
        c = Counter()
        for recipe in recipes:
            c.update([ingredient for ingredient in recipe if ingredient not in known])
            
        if p2:
            return certain
        else:
            return sum(c.values())
    
    def part2():
        certain = part1(True)
        certain = [pair[1] for pair in sorted(certain.items(),key=lambda x:x[0])]
        return ','.join(certain)

    return part1(),part2()

Day21()

(2659, 'rcqb,cltx,nrl,qjvvcvz,tsqpn,xhnk,tfqsb,zqzmzl')

In [18]:
def Day22(data=get_input(22),debug=False):
    P1 = 0
    P2 = 1
    
    def print(*args):
        if debug:
            __builtin__.print(*args)

    def get_players():
        player1,player2 = split_list(data, '')
        player1 = [int(i) for i in player1[1:]]
        player2 = [int(i) for i in player2[1:]]
        return player1,player2
    
    def part1():
        player1,player2 = get_players()
        while len(player1) > 0 and len(player2) > 0:
            top1,player1 = player1[0],player1[1:]
            top2,player2 = player2[0],player2[1:]
            if top1 > top2:
                player1 = player1 + [top1,top2]
            elif top2 > top1:
                player2 = player2 + [top2,top1]
            else:
                assert False

        winner = player1 if len(player1) > 0 else player2
        return score(winner)
    
    def score(deck):
        return sum([(idx+1)*val for idx,val in enumerate(reversed(deck))])
        
    MEMO = {}
    def get_result(d1,d2):
        key = ','.join(map(str,d1)) + '--' + ','.join(map(str,d2))
        if key in MEMO:
            return MEMO[key]
        
    def store_result(d1,d2,result):
        key = ','.join(map(str,d1)) + '--' + ','.join(map(str,d2))
        other_key = ','.join(map(str,d2)) + '--' + ','.join(map(str,d1))
        if result[0] == P1:
            other_result = (P2, result[1])
        else:
            other_result = (P1, result[1])
        MEMO[key] = result
        MEMO[other_key] = other_result
        return result
    
    gidx = 0
    def play_game(d1,d2):
        nonlocal gidx
        gidx += 1

        if get_result(d1,d2):
            res = get_result(d1,d2)
            print(f"MEMOIZED WINNER IS {res}")
            return res

        orig_d1 = d1.copy()
        orig_d2 = d2.copy()
        
        prev = []
        rounds = 0
        print(f"== GAME {gidx} ==")
        
        while True:
            if [d1,d2] in prev:
                print(f"Winner of Game {gidx} is player1 by infinite loop rule {prev} contains {[d1,d2]}")
                return store_result(orig_d1,orig_d2,(P1,d1))
            else:
                prev.append([d1,d2])
        
            if len(d1) == 0 or len(d2) == 0:
                winner_name = "player 1" if len(d2) == 0 else "player 2"
                print(f"Winner of Game {gidx} is {winner_name}")
                if len(d1) == 0:
                    return store_result(orig_d1,orig_d2,(P2,d2))
                else:
                    return store_result(orig_d1,orig_d2,(P1,d1))
            else:
                rounds += 1
                print(f"-- Round {rounds} (Game {gidx}) --")
                print(f"Player 1 deck {d1}")
                print(f"Player 2 deck {d2}")
                t1,d1 = d1[0],d1[1:]
                t2,d2 = d2[0],d2[1:]
                print(f"Player 1 plays {t1}")
                print(f"Player 2 plays {t2}")
                if len(d1) >= t1 and len(d2) >= t2:
                    sub_d1 = d1[:t1]
                    sub_d2 = d2[:t2]
                    # recurse
                    print(f"..recursing into sub-game")
                    winner,_ = play_game(sub_d1,sub_d2)
                    if winner == P1:
                        d1 = d1 + [t1,t2]
                    elif winner == P2:
                        d2 = d2 + [t2,t1]
                elif t1 > t2:
                    print(f"Player 1 wins round {rounds} of game {gidx}!")
                    d1 = d1 + [t1,t2]
                elif t2 > t1:
                    print(f"Player 2 wins round {rounds} of game {gidx}!")
                    d2 = d2 + [t2,t1]
                
    def part2():
        d1,d2 = get_players()
        _,deck = play_game(d1,d2)
        return score(deck)
        
    return part1(),part2()
ex = """Player 1:
9
2
6
3
1

Player 2:
5
8
4
7
10""".split("\n")
Day22()

(32162, 32534)

In [19]:
def Day23(data=get_input(23)):

    def print_nodes(node,NODES):
        output = []
        for _ in range(len(NODES)):
            output.append(node.val)
            node = node.next
        print(output)
    
    def part1():
        turns = 100
        cups = [int(d) for d in data[0]]
        min_cup = min(cups)
        max_cup = max(cups)
        nodes = play(turns,cups,min_cup,max_cup)
        node = nodes[1].next
        out = ""
        while True:
            out += str(node.val)
            node = node.next
            if node.val == 1: break
        return out
    
    def play(turns,cups,min_cup,max_cup):
        class Node: pass
        NODES = {v:Node() for v in cups}
        for idx,val in enumerate(cups):
            node = NODES[val]
            node.val = val
            if idx + 1 < len(cups):
                next_node = NODES[cups[idx+1]]
                node.next = next_node
            else:
                node.next = NODES[cups[0]]

        node = NODES[cups[0]]
        for turn in range(turns):
            pick_up = [node.next, node.next.next, node.next.next.next]
            pick_up_vals = [node.val for node in pick_up]
#             print(f"pick up {pick_up_vals}")

            dest_val = node.val
            dest_node = None
            while True:
                dest_val -= 1
                if dest_val < min_cup:
                    dest_val = max_cup
                if dest_val not in pick_up_vals:
                    dest_node = NODES[dest_val]
                    break

#             print(f"dest val: {dest_node.val}")
            node.next = pick_up[-1].next
            tmp_dest_node_next = dest_node.next
            dest_node.next = pick_up[0]
            pick_up[-1].next = tmp_dest_node_next
            node = node.next
        return NODES
    
    def part2():
        cups = [int(d) for d in data[0]]
        max_cup = max(cups)
        min_cup = 1
        for val in range(max_cup+1,1_000_000+1):
            cups.append(val)
        max_cup = max(cups)
        turns = 10_000_000
        nodes = play(turns,cups,min_cup,max_cup)
        node = nodes[1]
        return node.next.val * node.next.next.val
        
        
            
    
    return part1(),part2()

ex = """389125467""".split("\n")
print(f"Day 23: {Day23()}")

Day 23: ('45798623', 235551949822)


In [20]:
def Day24(data=get_input(24)):
    from collections import defaultdict

    WHITE = 0
    BLACK = 1
    
    DIRS = ['e','w','ne','nw','se','sw']
    
    def move(point,d):
        assert d in DIRS
        dx = None
        dy = None
        if d == 'e':
            dx = 1
            dy = 0
        elif d == 'w':
            dx = -1
            dy = 0
        elif d == 'ne':
            dy = -1
            if point[1] % 2 == 1:
                dx = 1
            else:
                dx = 0
        elif d == 'nw':
            dy = -1
            if point[1] % 2 == 1:
                dx = 0
            else:
                dx = -1
        elif d == 'se':
            dy = 1
            if point[1] % 2 == 1:
                dx = 1
            else:
                dx = 0
        elif d == 'sw':
            dy = 1
            if point[1] % 2 == 1:
                dx = 0
            else:
                dx = -1
        assert dx is not None
        assert dy is not None
        return (point[0]+dx,point[1]+dy)
    
    def to_coords(d_str):
        point = (0,0)
        while len(d_str) > 0:
            for d in DIRS:
                if d_str.startswith(d):
                    point = move(point,d)
                    d_str = d_str[len(d):]
                    break
        return point
    
    def neighbors(point, grid):
        return [grid[move(point,d)] for d in DIRS]
    
    def part1():
        grid = defaultdict(lambda:WHITE)
        for d_str in data:
            point = to_coords(d_str)
            if grid[point] == WHITE:
                grid[point] = BLACK
            else:
                grid[point] = WHITE
        return sum(grid.values())
    
    def tick(grid):
        next_grid = defaultdict(lambda:WHITE)
        for point in grid_points(grid.copy()):
            color = grid[point]
            ns = neighbors(point,grid)
            count_black = sum(ns)
            next_color = None
            if color == WHITE:
                if count_black == 2:
                    next_color = BLACK
                else:
                    next_color = WHITE
            else:
                if count_black == 0 or count_black > 2:
                    next_color = WHITE
                else:
                    next_color = BLACK
            assert next_color is not None
            next_grid[point] = next_color
        return next_grid
            
    def grid_points(grid):
        points = grid.keys()
        min_x = min([point[0] for point in points])
        max_x = max([point[0] for point in points])
        min_y = min([point[1] for point in points])
        max_y = max([point[1] for point in points])
        
#         print(f"x {min_x} -> {max_x}, y {min_y} -> {max_y}")

        
        for x in range(min_x-1,max_x+2):
            for y in range(min_y-1,max_y+2):
                yield (x,y)
    
    def part2():
        grid = defaultdict(lambda:WHITE)
        for d_str in data:
            point = to_coords(d_str)
            if grid[point] == WHITE:
                grid[point] = BLACK
            else:
                grid[point] = WHITE
        
        DAYS = 100
#         print(f"Day {0}: {sum(grid.values())}")
        for day in range(1,DAYS+1):
            grid = tick(grid)
#             print(f"Day {day}: {sum(grid.values())}")
            
        return sum(grid.values())
            
        
        
    return part1(),part2()

ex = """sesenwnenenewseeswwswswwnenewsewsw
neeenesenwnwwswnenewnwwsewnenwseswesw
seswneswswsenwwnwse
nwnwneseeswswnenewneswwnewseswneseene
swweswneswnenwsewnwneneseenw
eesenwseswswnenwswnwnwsewwnwsene
sewnenenenesenwsewnenwwwse
wenwwweseeeweswwwnwwe
wsweesenenewnwwnwsenewsenwwsesesenwne
neeswseenwwswnwswswnw
nenwswwsewswnenenewsenwsenwnesesenew
enewnwewneswsewnwswenweswnenwsenwsw
sweneswneswneneenwnewenewwneswswnese
swwesenesewenwneswnwwneseswwne
enesenwswwswneneswsenwnewswseenwsese
wnwnesenesenenwwnenwsewesewsesesew
nenewswnwewswnenesenwnesewesw
eneswnwswnwsenenwnwnwwseeswneewsenese
neswnwewnwnwseenwseesewsenwsweewe
wseweeenwnesenwwwswnew""".split("\n")
    
Day24()

(275, 3537)

In [85]:
def Day25(data=get_input(25)):
    data = list(map(int,data))
    
    def get_loops(pkey):
        subj = 7
        val = 1
        loops = 0
        nbr = 20201227
        val = 1
        while val != pkey:
            loops += 1
            val *= subj
            val = val % nbr
        return loops
    
    def transform(subj,loops=1):
        val = 1
        for _ in range(loops):
            val *= subj
            val = val % 20201227
        return val
    
    def part1():
        pkey1,pkey2 = data
        loops2 = get_loops(pkey2)
        return transform(pkey1, loops2)
    
    return part1()

Day25()

6198540